In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
train_features = pd.read_csv("training_set_features.csv")

In [ ]:
train_labels = pd.read_csv("training_set_labels.csv")

In [ ]:
test_features = pd.read_csv("test_set_features.csv")

In [ ]:
train_data = pd.merge(train_features, train_labels, on='respondent_id')

In [ ]:
X = train_data.drop(columns=['xyz_vaccine', 'seasonal_vaccine', 'respondent_id'])

In [ ]:
y_xyz = train_data['xyz_vaccine']
y_seasonal = train_data['seasonal_vaccine']

In [ ]:
test_data = test_features.drop(columns=['respondent_id'])

In [ ]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(include=[np.number]).columns.tolist()

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

In [ ]:
X_train, X_val, y_train_xyz, y_val_xyz, y_train_seasonal, y_val_seasonal = train_test_split(X, y_xyz, y_seasonal, test_size=0.2, random_state=42)

In [ ]:
model.fit(X_train, y_train_xyz)

In [ ]:
y_pred_xyz = model.predict_proba(X_val)[:, 1]
roc_auc_xyz = roc_auc_score(y_val_xyz, y_pred_xyz)

In [ ]:
print(f'ROC AUC for xyz_vaccine: {roc_auc_xyz}')

In [ ]:
model.fit(X_train, y_train_seasonal)

In [ ]:
y_pred_seasonal = model.predict_proba(X_val)[:, 1]
roc_auc_seasonal = roc_auc_score(y_val_seasonal, y_pred_seasonal)

In [ ]:
print(f'ROC AUC for seasonal_vaccine: {roc_auc_seasonal}')

In [ ]:
model.fit(X, y_xyz)
test_pred_xyz = model.predict_proba(test_data)[:, 1]

In [ ]:
model.fit(X, y_seasonal)
test_pred_seasonal = model.predict_proba(test_data)[:, 1]

In [ ]:
submission = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],
    'xyz_vaccine': test_pred_xyz,
    'seasonal_vaccine': test_pred_seasonal
})

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
import os
print("File saved in directory:", os.getcwd())